In [4]:
import boto3

# 建立與 sqs 的連線
sqs = boto3.client(
    'sqs',
    endpoint_url = 'http://cc104.sqs.local:9324',
    region_name = 'dummy_region',
    aws_access_key_id = 'dummy_access_key',
    aws_secret_access_key = 'dummy_secret_key',
    verify = False)

In [21]:
# Create a SQS queue
response = sqs.create_queue(
    QueueName='cc104',
    Attributes={
        'DelaySeconds': '60',
        'MessageRetentionPeriod': '86400'
    }
)

print(response['QueueUrl'])

# Create a SQS queue with long polling enabled
#response = sqs.create_queue(
#    QueueName='cc104_polling',
#    Attributes={'ReceiveMessageWaitTimeSeconds': '20'}
#)

#print(response['QueueUrl'])

http://cc104.sqs.local:9324/queue/cc104


In [24]:
# Get URL for SQS queue
response = sqs.get_queue_url(QueueName='cc104')

print(response['QueueUrl'])

# Get Attributes of queue
attrs = sqs.get_queue_attributes(
    QueueUrl = response['QueueUrl'],
    AttributeNames = ['All']
)
print (attrs['Attributes']['QueueArn'])

http://cc104.sqs.local:9324/queue/cc104
arn:aws:sqs:elasticmq:000000000000:cc104


In [14]:
# List SQS queues
response = sqs.list_queues()

print(response['QueueUrls'])

['http://cc104.sqs.local:9324/queue/queue1']


In [9]:
queue_url = 'http://cc104.sqs.local:9324/queue/cc104'

# Enable long polling on an existing SQS queue
#sqs.set_queue_attributes(
#    QueueUrl=queue_url,
#    Attributes={'ReceiveMessageWaitTimeSeconds': '20'}
#)

# Send message to SQS queue
response = sqs.send_message(
    QueueUrl=queue_url,
    DelaySeconds=10,
    MessageAttributes={
        'Title': {
            'DataType': 'String',
            'StringValue': 'The Whistler'
        },
        'Author': {
            'DataType': 'String',
            'StringValue': 'John Grisham'
        },
        'WeeksOn': {
            'DataType': 'Number',
            'StringValue': '6'
        }
    },
    MessageBody=(
        'Information about current NY Times fiction bestseller for '
        'week of 12/11/2016.'
    )
)

print(response['MessageId'])

60f82df7-0dad-41c4-a320-d1529a60d855


In [10]:
# Receive message from SQS queue
response = sqs.receive_message(
    QueueUrl = queue_url,
    AttributeNames=[
        'SentTimestamp'
    ],
    MaxNumberOfMessages=1,
    MessageAttributeNames=[
        'All'
    ],
    VisibilityTimeout=0,
    WaitTimeSeconds=0
)

# Long poll for message on provided SQS queue
#response = sqs.receive_message(
#    QueueUrl=queue_url,
#    AttributeNames=[
#        'SentTimestamp'
#    ],
#    MaxNumberOfMessages=1,
#    MessageAttributeNames=[
#        'All'
#    ],
#    WaitTimeSeconds=20
#)

print(response)

message = response['Messages'][0]
receipt_handle = message['ReceiptHandle']

In [11]:
# Delete received message from queue
sqs.delete_message(
    QueueUrl=queue_url,
    ReceiptHandle=receipt_handle
)
print('Received and deleted message: %s' % message)

Received and deleted message: {'MessageId': '60f82df7-0dad-41c4-a320-d1529a60d855', 'ReceiptHandle': '60f82df7-0dad-41c4-a320-d1529a60d855#77d2f16d-07e6-4c9b-9bad-cbe7406ed5c7', 'MD5OfBody': 'bbdc5fdb8be7251f5c910905db994bab', 'Body': 'Information about current NY Times fiction bestseller for week of 12/11/2016.', 'Attributes': {'SentTimestamp': '1547624437364'}, 'MD5OfMessageAttributes': 'd25a6aea97eb8f585bfa92d314504a92', 'MessageAttributes': {'Title': {'StringValue': 'The Whistler', 'DataType': 'String'}, 'Author': {'StringValue': 'John Grisham', 'DataType': 'String'}, 'WeeksOn': {'StringValue': '6', 'DataType': 'Number'}}}


In [13]:
# Delete SQS queue
sqs.delete_queue(QueueUrl=queue_url)

{'ResponseMetadata': {'RequestId': '00000000-0000-0000-0000-000000000000',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'akka-http/10.0.2',
   'date': 'Wed, 16 Jan 2019 07:49:43 GMT',
   'content-type': 'text/plain; charset=UTF-8',
   'content-length': '250'},
  'RetryAttempts': 0}}

In [16]:
dir(sqs)

['_PY_TO_OP_NAME',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache',
 '_client_config',
 '_convert_to_request_dict',
 '_emit_api_params',
 '_endpoint',
 '_exceptions',
 '_exceptions_factory',
 '_get_waiter_config',
 '_load_exceptions',
 '_loader',
 '_make_api_call',
 '_make_request',
 '_register_handlers',
 '_request_signer',
 '_response_parser',
 '_serializer',
 '_service_model',
 'add_permission',
 'can_paginate',
 'change_message_visibility',
 'change_message_visibility_batch',
 'create_queue',
 'delete_message',
 'delete_message_batch',
 'delete_queue',
 'exceptions',
 'generate_presigned_url',
 'get_paginator',
 'get_queue_attributes',
 'ge

In [19]:
help(sqs.get_queue_attributes)

Help on method get_queue_attributes in module botocore.client:

get_queue_attributes(*args, **kwargs) method of botocore.client.SQS instance
    Gets attributes for the specified queue.
    
     
    
    .. note::
    
       
    
      To determine whether a queue is `FIFO <http://docs.aws.amazon.com/AWSSimpleQueueService/latest/SQSDeveloperGuide/FIFO-queues.html>`__ , you can check whether ``QueueName`` ends with the ``.fifo`` suffix.
    
       
    
     
    
    Some actions take lists of parameters. These lists are specified using the ``param.n`` notation. Values of ``n`` are integers starting from 1. For example, a parameter list with two elements looks like this:
    
     
    
     ``&Attribute.1=first``  
    
     
    
     ``&Attribute.2=second``  
    
    
    
    See also: `AWS API Documentation <https://docs.aws.amazon.com/goto/WebAPI/sqs-2012-11-05/GetQueueAttributes>`_
    
    
    **Request Syntax** 
    ::
    
      response = client.get_queue_attributes(
